In [3]:
from pyspark.sql import SparkSession


spark = SparkSession\
    .builder \
    .appName("Twitter_Silver") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/10/18 23:07:11 WARN Utils: Your hostname, AnselmoHome resolves to a loopback address: 127.0.1.1; using 172.17.64.178 instead (on interface eth0)
24/10/18 23:07:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/10/18 23:07:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_tweet = spark.read.json("/home/anselmo/airflow_twitter/datalake/silver/twitter_datascience/tweet")

In [5]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [6]:
from pyspark.sql import functions as f

In [7]:
tweet_conversas = df_tweet.alias('tweet')\
                    .groupBy(f.to_date('created_at').alias('created_date'))\
                        .agg(
                            f.count_distinct('author_id').alias('n_tweets'),
                            f.sum('like_count').alias('n_like'),
                            f.sum('quote_count').alias('n_quote'),
                            f.sum('reply_count').alias('n_reply'),
                            f.sum('retweet_count').alias('n_retweet')
                            ).withColumn('weekday', f.date_format('created_date', 'E'))
                        

In [8]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2024-10-15|       9|   469|    385|    575|      364|    Tue|
+------------+--------+------+-------+-------+---------+-------+

